<a href="https://colab.research.google.com/github/Bill-Roa/Descargar-ERA5-Colab-Hidrologia/blob/main/era5_land_william.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Autor: Santiago Vega Guacaneme
# Fecha: 09/06/2024
# Descripción: Herramienta que descarga y visualiza variables oceanográficas del modelo de clima global ERA5. Descripción del modelo: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview
!pip install cdsapi
!pip install netCDF4
!pip install python-dotenv

import tkinter as tk
from tkinter import ttk
import cdsapi
import datetime
from netCDF4 import Dataset, num2date
import pandas as pd
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"


CoordenadaX = float(input("Coordenada en formato longitud (x): "))
CoordenadaY = float(input("Coordenada en formato latitud (y): "))
Año1 = int(input("Año inicial: "))
Año2 = int(input("Año final: "))
Variable = int(input("Variables: 1- Wind  2- Wave  3- Surface net solar radiation  4- 2m temperature  5- evaporation  "))

# Puntos con datos más cercanos a las coordenadas dadas
# (X1, Y1) (X2, Y1)
# (X1, Y2) (X2, Y2)
# Puntos = [Y1, X1, Y2, X2]
Puntos = [(int(CoordenadaY / 0.25) + 1) * 0.25, (int(CoordenadaX / 0.25) - 1) * 0.25, (int(CoordenadaY / 0.25)) * 0.25, (int(CoordenadaX / 0.25)) * 0.25]

# Distancia a los puntos con datos más cercanos a las coordenadas dadas
# (d1) (d2)
# (d3) (d4)
# Distancias = [d1, d2, d3, d4]
Distancias = [((CoordenadaX - Puntos[1]) ** 2 + (CoordenadaY - Puntos[0]) ** 2) ** 0.5, ((CoordenadaX - Puntos[3]) ** 2 + (CoordenadaY - Puntos[0]) ** 2) ** 0.5,
              ((CoordenadaX - Puntos[1]) ** 2 + (CoordenadaY - Puntos[2]) ** 2) ** 0.5, ((CoordenadaX - Puntos[3]) ** 2 + (CoordenadaY - Puntos[2]) ** 2) ** 0.5]

# Calculo del factor λ asociado a los puntos con datos más cercanos a las coordenadas dadas (Método de interpolación IDW)
# (λ1) (λ2)
# (λ3) (λ4)
# λ = [λ1, λ2, λ3, λ4]
λ = [(1/a**2)/sum([1/b**2 for b in Distancias]) for a in Distancias]

import cdsapi

TOKENS = {
    "1": ("WILLIAM", "fe398e0b-1ff0-464e-af86-4fe1c3869ba4"),
    "2": ("MARIA", "3d2eadcc-0af4-46c8-ba1a-69fbc41c0f75"),
    "3": ("ORLANDO", "e987229d-6303-4076-84e1-a02809d733f5")
}

opcion = input("Seleccione un usuario (1-WILLIAM, 2-MARIA, 3-ORLANDO): ")
usuario, token = TOKENS.get(opcion, (None, None))

if not usuario:
    print("Selección inválida. Saliendo...")
    exit()

c = cdsapi.Client(url="https://cds.climate.copernicus.eu/api", key=token)
print(f"Token de {usuario} seleccionado. Listo para descargar datos.")

for year in range(Año1, Año2+1):
    if Variable == 1:
        c.retrieve('reanalysis-era5-single-levels',
            {
                'variable': ['10m_u_component_of_wind', '10m_v_component_of_wind',],
                'year': str(year),
                'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',],
                'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',
                        '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',],
                'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00',],
                'area': Puntos,
                'data_format': 'netcdf',
                'product_type': 'reanalysis',
            },  r'wind_'+str(year)+'.nc')

    elif Variable == 2:
        c.retrieve('reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': ['mean_wave_direction', 'mean_wave_period', 'significant_height_of_combined_wind_waves_and_swell',],
                'year': str(year),
                'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',],
                'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',
                        '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',],
                'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00',],
                'area': [Puntos], #quitar corchetes
                'data_format': 'netcdf',
            },  r'wave_'+str(year)+'.nc')

    elif Variable == 3:
        c.retrieve('reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': ['surface_net_solar_radiation'],
                'year': str(year),
                'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',],
                'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',
                        '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',],
                'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00',],
                'area': [Puntos], #quitar corchetes
                'data_format': 'netcdf',
            },  r'surface_net_solar_radiation_'+str(year)+'.nc')

    elif Variable == 4:
        c.retrieve('reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': ['2m_temperature'],
                'year': str(year),
                'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',],
                'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',
                        '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',],
                'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00',],
                'area': [Puntos], #quitar corchetes
                'data_format': 'netcdf',
            },  r'2m_temperature_'+str(year)+'.nc')

    elif Variable == 5:
        c.retrieve('reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': ['evaporation'],
                'year': str(year),
                'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',],
                'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',
                        '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',],
                'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00',],
                'area': [Puntos], #quitar corchetes
                'data_format': 'netcdf',
            },  r'evaporation_'+str(year)+'.nc')

data = pd.DataFrame()


for year in range(Año1, Año2+1):
    if Variable == 1:
        file_in = Dataset(r'wind_'+str(year)+'.nc',format='NETCDF4')

        nctime = file_in.variables['valid_time'][:] # get values
        t_unit = file_in.variables['valid_time'].units # get unit  'days since 1950-01-01T00:00:00Z'
        t_cal = file_in.variables['valid_time'].calendar
        tvalue = num2date(nctime,units = t_unit,calendar = t_cal)
        str_time = [i.strftime('%Y-%m-%d %H:%M') for i in tvalue] # to display dates as string
        time = pd.DataFrame(str_time,columns=['Hora'])

        v10_data = file_in.variables['v10'][:]
        u10_data = file_in.variables['u10'][:]
        v10_1 = [v10_data[a,0,0]*λ[0]+v10_data[a,0,0]*λ[1]+v10_data[a,0,0]*λ[2]+v10_data[a,0,0]*λ[3] for a in range(len(v10_data))]
        u10_1 = [u10_data[a,0,0]*λ[0]+u10_data[a,0,0]*λ[1]+u10_data[a,0,0]*λ[2]+u10_data[a,0,0]*λ[3] for a in range(len(u10_data))]
        u10 = pd.DataFrame(u10_1,columns=['u10'])
        v10 = pd.DataFrame(v10_1,columns=['v10'])

        x = pd.concat([time, u10, v10], axis=1)
        data = pd.concat([data, x])

    elif Variable == 2:
        file_in = Dataset(r'wave_'+str(year)+'.nc',format='NETCDF4')

        nctime = file_in.variables['valid_time'][:] # get values
        t_unit = file_in.variables['valid_time'].units # get unit  'days since 1950-01-01T00:00:00Z'
        t_cal = file_in.variables['valid_time'].calendar
        tvalue = num2date(nctime,units = t_unit,calendar = t_cal)
        str_time = [i.strftime('%Y/%m/%d %H:%M') for i in tvalue] # to display dates as string
        time = pd.DataFrame(str_time,columns=['Hora'])

        mwd_data = file_in.variables['mwd'][:]
        mwp_data = file_in.variables['mwp'][:]
        swh_data = file_in.variables['swh'][:]
        mwd_1 = [mwd_data[a,0,0]*λ[0]+mwd_data[a,0,0]*λ[1]+mwd_data[a,0,0]*λ[2]+mwd_data[a,0,0]*λ[3] for a in range(len(mwd_data))]
        mwp_1 = [mwp_data[a,0,0]*λ[0]+mwp_data[a,0,0]*λ[1]+mwp_data[a,0,0]*λ[2]+mwp_data[a,0,0]*λ[3] for a in range(len(mwp_data))]
        swh_1 = [swh_data[a,0,0]*λ[0]+swh_data[a,0,0]*λ[1]+swh_data[a,0,0]*λ[2]+swh_data[a,0,0]*λ[3] for a in range(len(swh_data))]
        mwd = pd.DataFrame(mwd_1,columns=['Mean wave direction'])
        mwp = pd.DataFrame(mwp_1,columns=['Mean wave period'])
        swh = pd.DataFrame(swh_1,columns=['Significant height of combined wind waves and swell'])

        x = pd.concat([time, mwd, mwp, swh], axis=1)
        data = pd.concat([data, x])

    elif Variable == 3:
        file_in = Dataset(r'Surface_net_solar_radiation_'+str(year)+'.nc',format='NETCDF4')

        nctime = file_in.variables['valid_time'][:] # get values
        t_unit = file_in.variables['valid_time'].units # get unit  'days since 1950-01-01T00:00:00Z'
        t_cal = file_in.variables['valid_time'].calendar
        tvalue = num2date(nctime,units = t_unit,calendar = t_cal)
        str_time = [i.strftime('%Y/%m/%d %H:%M') for i in tvalue] # to display dates as string
        time = pd.DataFrame(str_time,columns=['Hora'])

        ssr_data = file_in.variables['ssr'][:]
        ssr_1 = [ssr_data[a,0,0]*λ[0]+ssr_data[a,0,0]*λ[1]+ssr_data[a,0,0]*λ[2]+ssr_data[a,0,0]*λ[3] for a in range(len(ssr_data))]
        ssr = pd.DataFrame(ssr_1,columns=['Surface net solar radiation'])
        x = pd.concat([time, ssr], axis=1)
        data = pd.concat([data, x])

    elif Variable == 4:
        file_in = Dataset(r'2m_temperature_'+str(year)+'.nc',format='NETCDF4')

        nctime = file_in.variables['valid_time'][:] # get values
        t_unit = file_in.variables['valid_time'].units # get unit  'days since 1950-01-01T00:00:00Z'
        t_cal = file_in.variables['valid_time'].calendar
        tvalue = num2date(nctime,units = t_unit,calendar = t_cal)
        str_time = [i.strftime('%Y/%m/%d %H:%M') for i in tvalue] # to display dates as string
        time = pd.DataFrame(str_time,columns=['Hora'])

        t2m_data = file_in.variables['t2m'][:]
        t2m_1 = [t2m_data[a,0,0]*λ[0]+t2m_data[a,0,0]*λ[1]+t2m_data[a,0,0]*λ[2]+t2m_data[a,0,0]*λ[3] for a in range(len(t2m_data))]
        t2m = pd.DataFrame(t2m_1,columns=['2m_temperature'])
        x = pd.concat([time, t2m-273.15], axis=1)
        data = pd.concat([data, x])

    elif Variable == 5:
        file_in = Dataset(r'evaporation_'+str(year)+'.nc',format='NETCDF4')

        nctime = file_in.variables['valid_time'][:] # get values
        t_unit = file_in.variables['valid_time'].units # get unit  'days since 1950-01-01T00:00:00Z'
        t_cal = file_in.variables['valid_time'].calendar
        tvalue = num2date(nctime,units = t_unit,calendar = t_cal)
        str_time = [i.strftime('%Y/%m/%d %H:%M') for i in tvalue] # to display dates as string
        time = pd.DataFrame(str_time,columns=['Hora'])

        e_data = file_in.variables['e'][:]
        e_1 = [e_data[a,0,0]*λ[0]+e_data[a,0,0]*λ[1]+e_data[a,0,0]*λ[2]+e_data[a,0,0]*λ[3] for a in range(len(e_data))]
        e = pd.DataFrame(e_1,columns=['evaporation'])
        x = pd.concat([time, e], axis=1)
        data = pd.concat([data, x])


# Solicitar al usuario los años y el nombre del archivo
file_name = input("Ingrese nombre del output ")

# Crear el archivo Excel con el nombre ingresado
datatoexcel = pd.ExcelWriter(rf'{file_name} {Año1}-{Año2}.xlsx')
data.to_excel(datatoexcel)
datatoexcel._save()

print(f'DataFrame se ha escrito en el archivo {file_name} {Año1}-{Año2}.xlsx exitosamente.')

# Descargar el archivo desde Google Colab
from google.colab import files
files.download(rf'{file_name} {Año1}-{Año2}.xlsx')

# Eliminar archivos .nc generados
for file in os.listdir():
    if file.endswith(".nc"):
        os.remove(file)




2025-02-13 15:38:16,694 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.legacy_api_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-02-13 15:38:16,697 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


Token de ORLANDO seleccionado. Listo para descargar datos.


2025-02-13 15:38:17,607 INFO Request ID is abf77c50-3950-4fc2-b7f0-980e14f0b549
INFO:datapi.legacy_api_client:Request ID is abf77c50-3950-4fc2-b7f0-980e14f0b549
2025-02-13 15:38:17,786 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-02-13 15:38:31,841 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been updated to successful


a87df339438267f0bab35d6de2f72f70.nc:   0%|          | 0.00/252k [00:00<?, ?B/s]

2025-02-13 15:38:34,487 INFO Request ID is 9496ebc2-99d2-4ee8-ad0b-40cd8bb9a875
INFO:datapi.legacy_api_client:Request ID is 9496ebc2-99d2-4ee8-ad0b-40cd8bb9a875
2025-02-13 15:38:34,654 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-02-13 15:38:43,700 INFO status has been updated to running
INFO:datapi.legacy_api_client:status has been updated to running
2025-02-13 15:38:48,925 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been updated to successful


...

2025-02-13 18:28:28,328 INFO Request ID is ac5325e3-df94-428c-b3fe-cc828d31228a
INFO:datapi.legacy_api_client:Request ID is ac5325e3-df94-428c-b3fe-cc828d31228a
2025-02-13 18:28:28,524 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-02-13 18:28:37,369 INFO status has been updated to running
INFO:datapi.legacy_api_client:status has been updated to running
2025-02-13 18:32:49,803 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been updated to successful


fff8856565fcce3efc8e3510fa417f83.nc:   0%|          | 0.00/252k [00:00<?, ?B/s]

Ingrese nombre del output evaporacion
DataFrame se ha escrito en el archivo evaporacion 1986-2024.xlsx exitosamente.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>